<a href="https://colab.research.google.com/github/IsaacSGray/L02twopOpt/blob/main/L04_Shelling_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [L04: Implementing the Shelling Model](https://docs.google.com/document/d/1NtGAKnsfwLRYMYrL74aMVMtTuQRJ1oDjvSFWlmMjpzs/edit?usp=sharing)
Written by Dr. Jan Pearce, Berea College

Complete the code by meeting all linked requirements and fixing all FIXMEs

## Your Name: Isaac Gray

**SUMMARY**: My initial design evolved quite drastically. Originally I wanted to do networkX, however with some experimenting and some asking around, matplotlib was much easier to use which has coloring functionality, plus we also have X,Y coordinates which we are able to 'swap' Around based on checks. We can also color agents based on values.



**PERSONNEL**: I worked with the TA, discussing some ideas, stub testing, also helping the TA in turn understanding the project requirement.

**CHALLENGES**: Descriptions of the largest challenges you overcame and what made them challenging.

One of the biggest challenges that faced me was testing this thing- it was hard to test it in small chunks considering that test firing one part at a time and seeing how it interacted with one another was hard to conceptualize. I did some doodling on whiteboard and paper to help find bounding

**INNOVATIONS**:I didn't do any extra innovations not required by the document.

**TESTING**: First as I developed my init function developing the data I need to keep persistent and eventually print at the end  of runtime as required by the lab document. Now after that I built development of my grid, and then devved and tested my checking of each node is satisfied, neighbors, and then swaps, then running until completion.

**ERRORS**: A list in bulleted form of all known errors and deficiencies.

Order of growth of his class is incredibly bad. I mean, anything about 60x50 ends up taking much longer  time to get a single grid.

**COMMENTS**: A paragraph or so of your own comments on and reactions to the Lab.

The lab was interesting, however given how tight the last month of all the due dates will be in this last month- its rough. I felt crunched by this project atop my capstone and HCC project, plus my calculus exam study period, plus the final exam for this course.

## Import Libraries

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import math
from urllib.request import urlopen
import random
import numpy as np

## The Schelling Model

In [ ]:

class Schelling:
    def __init__(self, width, height, empty_ratio, similarity_threshold):
      """
      init function which intializes all variables and data structures upon creation
      width:how many agents wide
      height:how many agents tall
      empty_ratio:#of agents whom are neutral, thus always satisfied
      similarity_threshhold:Ratio of neighbors who must be similar
      grid:2d array of agents
      agentsmoved:list of agents that moved
      """
        self.width = width
        self.height = height
        self.empty_ratio = empty_ratio
        self.similarity_threshold = similarity_threshold
        self.grid = self.initialize_grid()
        self.agentsmoved=[]

    def initialize_grid(self):
      """
      num empty:exact count of empty cells
      num agents:exact count of the number of agents, ignoring empty agents
      two types of agents, 1 and 2, agents of type 0 are neutral
      our grid  will make random choice for each agent type, for the entirety of num_agents
      our grid will extend and count neutral agents, '0'
      returns our numpy array of our grid, reshaping it by height and width,
      """
        # Create a grid with empty spaces and two types of agents
        num_empty = int(self.width * self.height * self.empty_ratio)
        num_agents = self.width * self.height - num_empty
        agent_types = [1, 2]  # Representing two different groups
        grid = [random.choice(agent_types) for _ in range(num_agents)]
        grid.extend([0] * num_empty)  # 0 represents empty space
        random.shuffle(grid)
        return np.array(grid).reshape(self.height, self.width)

    def is_satisfied(self, x, y):
        # Check if an agent is satisfied with its current location
        """
        agent type is the value stored at this x and y
        neighbors uses our built in get neighbors function, checking all direct neighbors
        Counts the number of similar neighbors to total neighbors
        """
        agent_type = self.grid[x, y]
        if agent_type == 0:  # Empty spaces are always satisfied
            return True

        neighbors = self.get_neighbors(x, y)
        similar_neighbors = sum(1 for neighbor in neighbors if neighbor == agent_type)
        total_neighbors = len(neighbors)

        if total_neighbors == 0:
            return True  # Satisfied if no neighbors

        similarity_ratio = similar_neighbors / total_neighbors
        return similarity_ratio >= self.similarity_threshold

    def get_neighbors(self, x, y):
        # Get the neighbors of an agent (including empty spaces)
        """
        This searches the neighbors, possible combination between a range of min and max y and x values
        """
        neighbors = []
        for i in range(max(0, x - 1), min(self.height, x + 2)):
            for j in range(max(0, y - 1), min(self.width, y + 2)):
                if (i, j) != (x, y):
                    neighbors.append(self.grid[i, j])
        return neighbors

    def update(self):
        # Move unsatisfied agents to empty locations
        """
        This searches every X for Y, and every Y for X to ensure we search the entire grid,verifies if its in is_satisfied , and then appends
        to the unsatisfied agents list
        randomly shuffles this list.
        Loops over it, verified if our x,y is not a neutral or 'empty' cell
        We build a list of all the empty cells, we check if any exist, and make random choice with new x and new y. We then append
        the x and y to the agents moved list.
        """
        unsatisfied_agents = []
        for x in range(self.height):
            for y in range(self.width):
                if not self.is_satisfied(x, y):
                    unsatisfied_agents.append((x, y))

        random.shuffle(unsatisfied_agents)
        for x, y in unsatisfied_agents:
            if self.grid[x, y] != 0:  # If not already empty
                empty_locations = [(i, j) for i in range(self.height)
                                  for j in range(self.width) if self.grid[i, j] == 0]
                if empty_locations:
                    new_x, new_y = random.choice(empty_locations)
                    self.grid[new_x, new_y], self.grid[x, y] = self.grid[x, y], self.grid[new_x, new_y]
                    if (x,y) not in self.agentsmoved:
                      self.agentsmoved.append((x,y))

    def plot(self):
        # Plot the grid using matplotlib
        """
        I use a color map which is applicable to colorblind people.
        Interpolation is a rendering setting which I fooled around with till I liked one for rendering the plot
        """
        plt.imshow(self.grid, cmap='viridis', interpolation='nearest')
        plt.title('Schelling Segregation Model')
        plt.show()

    def print_results(self):
      """
      Prints the results required by Lab04, can be called before and after runtime of the model.
      """
      total_segregation=0
      for x in range(self.height):
        for y in range(self.width):
          if self.is_satisfied(x,y):
            total_segregation+=1
      total_segregation=total_segregation/(self.height*self.width)
      redagents=0
      for x in range(self.height):
        for y in range(self.width):
          if self.grid[x,y]==1:
            redagents+=1
      blueagents=0
      for x in range(self.height):
        for y in range(self.width):
          if self.grid[x,y]==2:
            blueagents+=1
      print("Agents Moved:",len(self.agentsmoved))
      print("Total Segregation:",total_segregation)
      print("Number of Red Agents",redagents)
      print("Number of Blue Agents",blueagents)

# Example usage
schelling = Schelling(width=50, height=60, empty_ratio=0.40, similarity_threshold=0.3)
schelling.plot()
schelling.print_results()
for _ in range(100):  # Run for 10 iterations
    schelling.update()
    schelling.plot()
schelling.print_results()

## Integrity statement

Please briefly describe all references you used, all help you received and all help you gave to others in completing this assignment. Be sure to say that you got no help if you got none.

FIXME